### `Introduction`
This notebook is an example of weighted least squares (WLS)-based GNSS single point positioning using only the provided `device_gnss.csv` and Kalman smoother. The WLS code is based on @junkoda 's [code](https://www.kaggle.com/code/junkoda/deriving-baseline-wls-positions-in-progress) and [code](https://www.kaggle.com/code/junkoda/wls-velocity-estimation-from-doppler-shift) and @taroz1461[code](https://www.kaggle.com/code/taroz1461/carrier-smoothing-robust-wls-kalman-smoother). Thank you very much!

The major features of this notebook are as follows
- Speedup of WSL computation by linearization and Jacobi matrix calculation
- Application of robust optimization methods for outliers
- [Carrier smoothing](https://gssc.esa.int/navipedia/index.php/Carrier-smoothing_of_code_pseudoranges) of noisy pseudorange with carrier phase (ADR) ovservation
- Estimated WLS position and velocity are integrated using the simple Kalman Smoother
- Parameter tuning for carrier phase jump threshold and pseudo range jump threshold

This notebook is a basic pseudorange based WSL with minimal processing of position and velocity integration, so there is a lot of room for improvement!

In [78]:
import math
import numpy as np 
import pandas as pd
from IPython.display import Image
from IPython.core.display import HTML 
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy import signal

import pymap3d as pm
import pymap3d.vincenty as pmv
import plotly.express as px
import pyproj
import matplotlib.pyplot as plt
import glob as gl
import scipy.optimize
from tqdm.auto import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.spatial import distance
import pyproj as proj
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# DF view config  
pd.options.display.max_columns = 50

# Constants
CLIGHT = 299_792_458   # speed of light (m/s)
RE_WGS84 = 6_378_137   # earth semimajor axis (WGS84) (m)
OMGE = 7.2921151467E-5  # earth angular velocity (IS-GPS) (rad/s)

### `Satellite Selection`

In [13]:
# Satellite selection using carrier frequency error, elevation angle, and C/N0
def satellite_selection(df, column):
    """
    Args:
        df : DataFrame from device_gnss.csv
        column : Column name
    Returns:
        df: DataFrame with eliminated satellite signals
    """
    idx = df[column].notnull()
    idx &= df['CarrierErrorHz'] < 2.0e6  # carrier frequency error (Hz)
    idx &= df['SvElevationDegrees'] > 15.0  # elevation angle (deg)
    idx &= df['Cn0DbHz'] > 10.0  # C/N0 (dB-Hz)
    idx &= df['MultipathIndicator'] == 0 # Multipath flag

    return df[idx]

### `Pseudorange/Doppler Residuals and Jacobian`

In [14]:
# Compute line-of-sight vector from user to satellite
def los_vector(xusr, xsat):
    """
    Args:
        xusr : user position in ECEF (m)
        xsat : satellite position in ECEF (m)
    Returns:
        u: unit line-of-sight vector in ECEF (m)
        rng: distance between user and satellite (m)
    """
    u = xsat - xusr
    rng = np.linalg.norm(u, axis=1).reshape(-1, 1)
    u /= rng
    
    return u, rng.reshape(-1)


# Compute Jacobian matrix
def jac_pr_residuals(x, xsat, pr, W):
    """
    Args:
        x : current position in ECEF (m)
        xsat : satellite position in ECEF (m)
        pr : pseudorange (m)
        W : weight matrix
    Returns:
        W*J : Jacobian matrix
    """
    u, _ = los_vector(x[:3], xsat)
    J = np.hstack([-u, np.ones([len(pr), 1])])  # J = [-ux -uy -uz 1]

    return W @ J


# Compute pseudorange residuals
def pr_residuals(x, xsat, pr, W):
    """
    Args:
        x : current position in ECEF (m)
        xsat : satellite position in ECEF (m)
        pr : pseudorange (m)
        W : weight matrix
    Returns:
        residuals*W : pseudorange residuals
    """
    u, rng = los_vector(x[:3], xsat)

    # Approximate correction of the earth rotation (Sagnac effect) often used in GNSS positioning
    rng += OMGE * (xsat[:, 0] * x[1] - xsat[:, 1] * x[0]) / CLIGHT

    # Add GPS L1 clock offset
    residuals = rng - (pr - x[3])

    return residuals @ W


# Compute Jacobian matrix
def jac_prr_residuals(v, vsat, prr, x, xsat, W):
    """
    Args:
        v : current velocity in ECEF (m/s)
        vsat : satellite velocity in ECEF (m/s)
        prr : pseudorange rate (m/s)
        x : current position in ECEF (m)
        xsat : satellite position in ECEF (m)
        W : weight matrix
    Returns:
        W*J : Jacobian matrix
    """
    u, _ = los_vector(x[:3], xsat)
    J = np.hstack([-u, np.ones([len(prr), 1])])

    return np.dot(W, J)


# Compute pseudorange rate residuals
def prr_residuals(v, vsat, prr, x, xsat, W):
    """
    Args:
        v : current velocity in ECEF (m/s)
        vsat : satellite velocity in ECEF (m/s)
        prr : pseudorange rate (m/s)
        x : current position in ECEF (m)
        xsat : satellite position in ECEF (m)
        W : weight matrix
    Returns:
        residuals*W : pseudorange rate residuals
    """
    u, rng = los_vector(x[:3], xsat)
    rate = np.sum((vsat-v[:3])*u, axis=1) \
          + OMGE / CLIGHT * (vsat[:, 1] * x[0] + xsat[:, 1] * v[0]
                           - vsat[:, 0] * x[1] - xsat[:, 0] * v[1])

    residuals = rate - (prr - v[3])

    return residuals @ W

###    `Carrier Smoothing`
Smoothing noisy pseudoranges with accurate (but absolute distance bias exists) carrier phase. Note that this is a post-processing method. The average absolute distance is added to continuously observed carrier phase.

In [15]:
# Carrier smoothing of pseudorange
def carrier_smoothing(gnss_df):
    """
    Args:
        df : DataFrame from device_gnss.csv
    Returns:
        df: DataFrame with carrier-smoothing pseudorange 'pr_smooth'
    """
    carr_th = 1.6 # carrier phase jump threshold [m] 2->1.5 (best)->1.0
    pr_th =  20.0 # pseudorange jump threshold [m] 20->15

    prsmooth = np.full_like(gnss_df['RawPseudorangeMeters'], np.nan)
    # Loop for each signal
    for (i, (svid_sigtype, df)) in enumerate((gnss_df.groupby(['Svid', 'SignalType']))):
        df = df.replace(
            {'AccumulatedDeltaRangeMeters': {0: np.nan}})  # 0 to NaN

        # Compare time difference between pseudorange/carrier with Doppler
        drng1 = df['AccumulatedDeltaRangeMeters'].diff() - df['PseudorangeRateMetersPerSecond']
        drng2 = df['RawPseudorangeMeters'].diff() - df['PseudorangeRateMetersPerSecond']

        # Check cycle-slip
        slip1 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**1) != 0  # reset flag
        slip2 = (df['AccumulatedDeltaRangeState'].to_numpy() & 2**2) != 0  # cycle-slip flag
        slip3 = np.fabs(drng1.to_numpy()) > carr_th # Carrier phase jump
        slip4 = np.fabs(drng2.to_numpy()) > pr_th # Pseudorange jump

        idx_slip = slip1 | slip2 | slip3 | slip4
        idx_slip[0] = True

        # groups with continuous carrier phase tracking
        df['group_slip'] = np.cumsum(idx_slip)

        # Psudorange - carrier phase
        df['dpc'] = df['RawPseudorangeMeters'] - df['AccumulatedDeltaRangeMeters']

        # Absolute distance bias of carrier phase
        meandpc = df.groupby('group_slip')['dpc'].mean()
        df = df.merge(meandpc, on='group_slip', suffixes=('', '_Mean'))

        # Index of original gnss_df
        idx = (gnss_df['Svid'] == svid_sigtype[0]) & (
            gnss_df['SignalType'] == svid_sigtype[1])

        # Carrier phase + bias
        prsmooth[idx] = df['AccumulatedDeltaRangeMeters'] + df['dpc_Mean']

    # If carrier smoothing is not possible, use original pseudorange
    idx_nan = np.isnan(prsmooth)
    prsmooth[idx_nan] = gnss_df['RawPseudorangeMeters'][idx_nan]
    gnss_df['pr_smooth'] = prsmooth

    return gnss_df

### `Score Computation`

In [16]:
# Compute distance by Vincenty's formulae
def vincenty_distance(llh1, llh2):
    """
    Args:
        llh1 : [latitude,longitude] (deg)
        llh2 : [latitude,longitude] (deg)
    Returns:
        d : distance between llh1 and llh2 (m)
    """
    d, az = np.array(pmv.vdist(llh1[:, 0], llh1[:, 1], llh2[:, 0], llh2[:, 1]))

    return d


# Compute score
def calc_score(llh, llh_gt):
    """
    Args:
        llh : [latitude,longitude] (deg)
        llh_gt : [latitude,longitude] (deg)
    Returns:
        score : (m)
    """
    d = vincenty_distance(llh, llh_gt)
    score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])

    return score

### `Robust WLS`
I used `soft_l1` loss function. It is robust, but its computation speed is considerably slower than that of ordinary least squares...

In [17]:
# GNSS single point positioning using pseudorange
def point_positioning(gnss_df):
    # Add nominal frequency to each signal
    # Note: GLONASS is an FDMA signal, so each satellite has a different frequency
    CarrierFrequencyHzRef = gnss_df.groupby(['Svid', 'SignalType'])[
        'CarrierFrequencyHz'].median()
    gnss_df = gnss_df.merge(CarrierFrequencyHzRef, how='left', on=[
                            'Svid', 'SignalType'], suffixes=('', 'Ref'))
    gnss_df['CarrierErrorHz'] = np.abs(
        (gnss_df['CarrierFrequencyHz'] - gnss_df['CarrierFrequencyHzRef']))

    # Carrier smoothing
    gnss_df = carrier_smoothing(gnss_df)

    # GNSS single point positioning
    utcTimeMillis = gnss_df['utcTimeMillis'].unique()
    nepoch = len(utcTimeMillis)
    x0 = np.zeros(4)  # [x,y,z,tGPSL1]
    v0 = np.zeros(4)  # [vx,vy,vz,dtGPSL1]
    x_wls = np.full([nepoch, 3], np.nan)  # For saving position
    v_wls = np.full([nepoch, 3], np.nan)  # For saving velocity

    # Loop for epochs
    for i, (t_utc, df) in enumerate(tqdm(gnss_df.groupby('utcTimeMillis'), total=nepoch)):
        # Valid satellite selection
        df_pr = satellite_selection(df, 'pr_smooth')
        df_prr = satellite_selection(df, 'PseudorangeRateMetersPerSecond')

        # Corrected pseudorange/pseudorange rate
        pr = (df_pr['pr_smooth'] + df_pr['SvClockBiasMeters'] - df_pr['IsrbMeters'] -
              df_pr['IonosphericDelayMeters'] - df_pr['TroposphericDelayMeters']).to_numpy()
        prr = (df_prr['PseudorangeRateMetersPerSecond'] +
               df_prr['SvClockDriftMetersPerSecond']).to_numpy()

        # Satellite position/velocity
        xsat_pr = df_pr[['SvPositionXEcefMeters', 'SvPositionYEcefMeters',
                         'SvPositionZEcefMeters']].to_numpy()
        xsat_prr = df_prr[['SvPositionXEcefMeters', 'SvPositionYEcefMeters',
                           'SvPositionZEcefMeters']].to_numpy()
        vsat = df_prr[['SvVelocityXEcefMetersPerSecond', 'SvVelocityYEcefMetersPerSecond',
                       'SvVelocityZEcefMetersPerSecond']].to_numpy()

        # Weight matrix for peseudorange/pseudorange rate
        Wx = np.diag(1 / df_pr['RawPseudorangeUncertaintyMeters'].to_numpy())
        Wv = np.diag(1 / df_prr['PseudorangeRateUncertaintyMetersPerSecond'].to_numpy())

        # Robust WLS requires accurate initial values for convergence,
        # so perform normal WLS for the first time
        if len(df_pr) >= 4:
            # Normal WLS
            if np.all(x0 == 0):
                opt = scipy.optimize.least_squares(
                    pr_residuals, x0, jac_pr_residuals, args=(xsat_pr, pr, Wx))
                x0 = opt.x 
            # Robust WLS for position estimation
            opt = scipy.optimize.least_squares(
                 pr_residuals, x0, jac_pr_residuals, args=(xsat_pr, pr, Wx), loss='soft_l1')
            if opt.status < 1 or opt.status == 2:
                 print(f'i = {i} position lsq status = {opt.status}')
            else:
                 x_wls[i, :] = opt.x[:3]
                 x0 = opt.x
                 
        # Velocity estimation
        if len(df_prr) >= 4:
            if np.all(v0 == 0): # Normal WLS
                opt = scipy.optimize.least_squares(
                    prr_residuals, v0, jac_prr_residuals, args=(vsat, prr, x0, xsat_prr, Wv))
                v0 = opt.x
            # Robust WLS for velocity estimation
            opt = scipy.optimize.least_squares(
                prr_residuals, v0, jac_prr_residuals, args=(vsat, prr, x0, xsat_prr, Wv), loss='soft_l1')
            if opt.status < 1:
                print(f'i = {i} velocity lsq status = {opt.status}')
            else:
                v_wls[i, :] = opt.x[:3]
                v0 = opt.x

    return utcTimeMillis, x_wls, v_wls

### `Outlier Detection and Interpolation`

In [18]:
# Simple outlier detection and interpolation
def exclude_interpolate_outlier(x_wls, v_wls):
    # Up velocity threshold
    v_up_th = 2.0 # m/s

    # Coordinate conversion
    x_llh = np.array(pm.ecef2geodetic(x_wls[:, 0], x_wls[:, 1], x_wls[:, 2])).T
    v_enu = np.array(pm.ecef2enuv(
        v_wls[:, 0], v_wls[:, 1], v_wls[:, 2], x_llh[0, 0], x_llh[0, 1])).T

    # Up velocity jump detection
    # Cars don't jump suddenly!
    idx_v_out = np.abs(v_enu[:, 2]) > v_up_th
    v_wls[idx_v_out, :] = np.nan
    
    # Interpolate NaNs at beginning and end of array
    x_df = pd.DataFrame({'x': x_wls[:, 0], 'y': x_wls[:, 1], 'z': x_wls[:, 2]})
    x_df = x_df.interpolate(limit_area='outside', limit_direction='both')
    
    # Interpolate all NaN data
    v_df = pd.DataFrame({'x': v_wls[:, 0], 'y': v_wls[:, 1], 'z': v_wls[:, 2]})
    v_df = v_df.interpolate(limit_area='outside', limit_direction='both')
    v_df = v_df.interpolate('spline', order=3)

    return x_df.to_numpy(), v_df.to_numpy()

### `Kalman Smoother`
Application of Kalman smoother; integration of velocity and position obtained by WLS.
The covariance matrix is a fixed value.

In [19]:
# Kalman filter
def Kalman_filter(zs, us, phone):
    # Parameters
    # I don't know why only XiaomiMi8 seems to be inaccurate ... 
    sigma_v = 0.6 if phone == 'XiaomiMi8' else 0.1 # velocity SD m/s
    sigma_x = 5.0  # position SD m
    sigma_mahalanobis = 30.0 # Mahalanobis distance for rejecting innovation
    
    n, dim_x = zs.shape
    F = np.eye(3)  # Transition matrix
    Q = sigma_v**2 * np.eye(3)  # Process noise

    H = np.eye(3)  # Measurement function
    R = sigma_x**2 * np.eye(3)  # Measurement noise

    # Initial state and covariance
    x = zs[0, :3].T  # State
    P = sigma_x**2 * np.eye(3)  # State covariance
    I = np.eye(dim_x)

    x_kf = np.zeros([n, dim_x])
    P_kf = np.zeros([n, dim_x, dim_x])

    # Kalman filtering
    for i, (u, z) in enumerate(zip(us, zs)):
        # First step
        if i == 0:
            x_kf[i] = x.T
            P_kf[i] = P
            continue

        # Prediction step
        x = F @ x + u.T
        P = (F @ P) @ F.T + Q

        # Check outliers for observation
        d = distance.mahalanobis(z, H @ x, np.linalg.pinv(P))

        # Update step
        if d < sigma_mahalanobis:
            y = z.T - H @ x
            S = (H @ P) @ H.T + R
            K = (P @ H.T) @ np.linalg.inv(S)
            x = x + K @ y
            P = (I - (K @ H)) @ P
        else:
            # If no observation update is available, increase covariance
            P += 10**2*Q

        x_kf[i] = x.T
        P_kf[i] = P

    return x_kf, P_kf


# Forward + backward Kalman filter and smoothing
def Kalman_smoothing(x_wls, v_wls, phone):
    n, dim_x = x_wls.shape

    # Forward
    v = np.vstack([np.zeros([1, 3]), (v_wls[:-1, :] + v_wls[1:, :])/2])
    x_f, P_f = Kalman_filter(x_wls, v, phone)

    # Backward
    v = -np.flipud(v_wls)
    v = np.vstack([np.zeros([1, 3]), (v[:-1, :] + v[1:, :])/2])
    x_b, P_b = Kalman_filter(np.flipud(x_wls), v, phone)

    # Smoothing
    x_fb = np.zeros_like(x_f)
    P_fb = np.zeros_like(P_f)
    for (f, b) in zip(range(n), range(n-1, -1, -1)):
        P_fi = np.linalg.inv(P_f[f])
        P_bi = np.linalg.inv(P_b[b])

        P_fb[f] = np.linalg.inv(P_fi + P_bi)
        x_fb[f] = P_fb[f] @ (P_fi @ x_f[f] + P_bi @ x_b[b])

    return x_fb, x_f, np.flipud(x_b)

### `Train Data`

In [39]:
# Target course/phone
path = '../data/2021-07-19-US-MTV-1/GooglePixel4'

drive, phone = path.split('/')[-2:]

# Read data
gnss_df = pd.read_csv(f'{path}/device_gnss.csv')  # GNSS data
gt_df = pd.read_csv(f'{path}/ground_truth.csv')  # ground truth

# Point positioning
utc, x_wls, v_wls = point_positioning(gnss_df)

# Exclude velocity outliers
x_wls, v_wls = exclude_interpolate_outlier(x_wls, v_wls)

# Kalman smoothing
x_kf, _, _ = Kalman_smoothing(x_wls, v_wls, phone)

# Convert to latitude and longitude
llh_wls = np.array(pm.ecef2geodetic(x_wls[:, 0], x_wls[:, 1], x_wls[:, 2])).T
llh_kf = np.array(pm.ecef2geodetic(x_kf[:, 0], x_kf[:, 1], x_kf[:, 2])).T

# # Baseline
# x_bl = gnss_df.groupby('TimeNanos')[
#     ['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']].mean().to_numpy()
# llh_bl = np.array(pm.ecef2geodetic(x_bl[:, 0], x_bl[:, 1], x_bl[:, 2])).T

# # Ground truth
# llh_gt = gt_df[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()

# # Distance from ground truth
# vd_bl = vincenty_distance(llh_bl, llh_gt)
# vd_wls = vincenty_distance(llh_wls, llh_gt)
# vd_kf = vincenty_distance(llh_kf, llh_gt)

# # Score
# score_bl = calc_score(llh_bl, llh_gt)
# score_wls = calc_score(llh_wls, llh_gt)
# score_kf = calc_score(llh_kf[:-1, :], llh_gt[:-1, :])

# print(f'Score Baseline   {score_bl:.4f} [m]')
# print(f'Score Robust WLS {score_wls:.4f} [m]')
# print(f'Score KF         {score_kf:.4f} [m]')

# # Plot distance error
# plt.figure(figsize=(16,8))
# plt.title('Distance error')
# plt.ylabel('Distance error [m]')
# plt.plot(vd_bl, label=f'Baseline, Score: {score_bl:.4f} m')
# plt.plot(vd_wls, label=f'Robust WLS, Score: {score_wls:.4f} m')
# plt.plot(vd_kf, label=f'Robust WLS + KF, Score: {score_kf:.4f} m')
# plt.legend()
# plt.grid()
# plt.ylim([-1, 5])

# # Compute velocity error
# speed_wls = np.linalg.norm(v_wls[:, :3], axis=1)
# speed_gt = gt_df['SpeedMps'].to_numpy()
# speed_rmse = np.sqrt(np.sum((speed_wls-speed_gt)**2)/len(speed_gt))

# # Plot velocity error
# plt.figure(figsize=(16,8))
# plt.title('Speed error')
# plt.ylabel('Speed Error [m/s]')
# plt.plot(speed_wls - speed_gt, label=f'Speed RMSE: {speed_rmse:.4f} m')
# plt.legend()
# plt.grid()
# plt.ylim([-1, 1])


  0%|          | 0/1896 [00:00<?, ?it/s]

### `Mapping`

In [59]:
llh_bl_df = pd.DataFrame(llh_bl, columns = ['lat', 'lon', 'Noise'])
llh_wls_df = pd.DataFrame(llh_wls, columns = ['lat', 'lon', 'Noise'])
llh_kf_df = pd.DataFrame(llh_kf, columns = ['lat', 'lon', 'Noise'])

gt_df.rename(columns = {'LatitudeDegrees':'lat', 'LongitudeDegrees':'lon'}, inplace = True)

all_tracks = pd.concat([gt_df, llh_bl_df, llh_wls_df, llh_kf_df])
all_tracks  =all_tracks[['lat', 'lon']]
all_tracks['Name'] = np.repeat(['gt','bl', 'wls','kf'],gt_df.shape[0])

fig = px.line_mapbox(all_tracks, lat="lat", lon="lon",
                        color="Name", labels="Name",
                        zoom=12, center={"lat":37.45, "lon":-122.27},
                        height=600, width=800)
fig = px.line_mapbox(all_tracks, lat="lat", lon="lon",
                        color="Name", labels="Name",
                        zoom=12, center={"lat":37.45, "lon":-122.27},
                        height=600, width=800)
fig.update_layout(mapbox_style='stamen-terrain')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

### `Adding New Features` (Chad)

In [64]:
def convert_wgs_to_utm(lon, lat):
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+ utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code

def add_features(df):
    # setup your projections
    utm_code = convert_wgs_to_utm(df['lon'].mean(), df['lat'].mean())
    crs_wgs = proj.Proj(init='epsg:4326') # assuming using WGS84
    crs_utm = proj.Proj(init='epsg:{0}'.format(utm_code))
    x, y = np.multiply(proj.transform(crs_wgs, crs_utm, df['lon'], df['lat']),0.001)

    # Velocity and Speed
    x_t = np.gradient(x)
    y_t = np.gradient(y)

    vel = np.array([ [x_t[i], y_t[i]] for i in range(x_t.size)])
    
    speed = np.sqrt(x_t * x_t + y_t * y_t)
    
    tangent = np.array([1/speed] * 2).transpose() * vel

    # Curvature
    ss_t = np.gradient(speed)
    xx_t = np.gradient(x_t)
    yy_t = np.gradient(y_t)
    curvature = (xx_t * y_t - x_t * yy_t) / (x_t * x_t + y_t * y_t)**1.5
    xxyy = np.sqrt(xx_t * xx_t + yy_t * yy_t)
    
    # Accelaration
    tangent_x = tangent[:, 0]
    tangent_y = tangent[:, 1]

    deriv_tangent_x = np.gradient(tangent_x)
    deriv_tangent_y = np.gradient(tangent_y)

    dT_dt = np.array([ [deriv_tangent_x[i], deriv_tangent_y[i]] for i in range(deriv_tangent_x.size)])

    length_dT_dt = np.sqrt(deriv_tangent_x * deriv_tangent_x + deriv_tangent_y * deriv_tangent_y)

    normal = np.array([1/length_dT_dt] * 2).transpose() * dT_dt

    t_component = np.array([ss_t] * 2).transpose()
    n_component = np.array([curvature*ss_t * ss_t] * 2).transpose()
    accel = t_component * tangent + n_component * normal
    
    curvature[curvature>10]=5
    curvature[curvature<-10]=-5
    curvature = signal.medfilt(1/(1+np.exp(-signal.medfilt(curvature,3))),7)
    df['curve']= curvature
    df['speed']= signal.medfilt(speed,3)
    df['acc'] = signal.medfilt(np.linalg.norm(accel, axis=1),3)
    df['gas']= signal.medfilt(ss_t,3)
    df['dir_x'] = x_t / speed
    df['dir_y'] = y_t / speed
    # df['dir_x_t'] = xx_t / xxyy
    # df['dir_y_t'] = yy_t / xxyy
  
    return df

In [27]:
path = '../data/2021-07-19-US-MTV-1/GooglePixel4'
llh_kf_df = pd.DataFrame(llh_kf, columns = ['lat', 'lon', 'Noise'])
llh_kf_df.to_csv(f'{path}/llh_kf_df.csv', index=False)

In [28]:
path = '../data/2021-07-19-US-MTV-1/GooglePixel4'
llh_kf_df = pd.read_csv(f'{path}/llh_kf_df.csv')
gt_df = pd.read_csv(f'{path}/ground_truth.csv')  # ground truth
llh_gt = gt_df[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()

In [39]:
llh_kf_df = add_features(llh_kf_df)

### `Particle Filter`

In [186]:
# Getting ENU Coordinates with Velocity and Angle / Heading 

def ecef_to_enu(x_wls, llh_wls):

    # Converting ecef coordinates to ENU 
    enu_df = np.array(pm.ecef2enu(x_wls[:,0], x_wls[:,1], x_wls[:,2], llh_wls[:,0], llh_wls[:,1], llh_wls[:,2])).T    # east, north, up 

    # speed = np.sqrt(ew_t * ew_t + ns_t * ns_t)

In [189]:
ecef_to_enu(x_wls, llh_wls)

# Velocity (u) and Speed 
ew_t = np.gradient(enu_df[:,0]).reshape(len(enu_df),1)
ns_t = np.gradient(enu_df[:,1]).reshape(len(enu_df),1)

u = np.array([ [ew_t[i], ns_t[i]] for i in range(ew_t.size) ])

In [195]:
# Creating a uniform and Gaussian distribution of particles over a region 

from numpy.random import uniform

def create_uniform_particles(ew_range, ns_range, u_range, N):
    particles = np.empty((N, 3))
    particles[:, 0] = uniform(ew_range[0], ew_range[1], size=N)
    particles[:, 1] = uniform(ns_range[0], ns_range[1], size=N)
    particles[:, 2] = uniform(u_range[0], u_range[1], size=N)
    return particles

def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 3))
    particles[:, 0] = mean[0] + (randn(N) * std[0])
    particles[:, 1] = mean[1] + (randn(N) * std[1])
    particles[:, 2] = mean[2] + (randn(N) * std[2])
    return particles


# Predicting system state (location) after movement 

def predict(particles, u, std):
    """ move according to control input u (heading, velocity)
    with noise Q (std heading, std velocity)`"""

    N = len(particles)

    # move in the (noisy) commanded direction
    particles[:, 0] += u[0] + (randn(N) * std[1])
    particles[:, 1] += u[1] + (randn(N) * std[1])


# Update step (particle weights based on distance from observed location): 

def update(particles, weights, z, R, enu_df):
    for i, obs_location in enumerate(enu_df[:,0:2]):
        distance = np.linalg.norm(particles[:, 0:2] - obs_location, axis=1)
        weights *= scipy.stats.norm(distance, R).pdf(z[i])  # importance density 

    weights += 1.e-300      # avoid round-off to zero
    weights /= sum(weights) # normalize


# Computing system state (location) estimate 

def estimate(particles, weights):
    """returns mean and variance of the weighted particles"""

    pos = particles[:, 0:2]
    mean = np.average(pos, weights=weights, axis=0)
    var  = np.average((pos - mean)**2, weights=weights, axis=0)
    return mean, var


# Resampling 

def simple_resample(particles, weights):
    N = len(particles)
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1. # avoid round-off error
    indexes = np.searchsorted(cumulative_sum, random(N))

    # resample according to indexes
    particles[:] = particles[indexes]
    weights.fill(1.0 / N)

In [199]:
from filterpy.monte_carlo import systematic_resample
from numpy.linalg import norm
from numpy.random import randn
import scipy.stats

# range values 
ew_max, ns_max, ud_max = enu_df.max(axis=0)
print(ew_max, ns_max, ud_max)

ew_min, ns_min, ud_min = enu_df.min(axis=0)
print(ew_min, ns_min, ud_min)


def run_pf1(N, iters=5, sensor_std_err=.1, 
            do_plot=True, plot_particles=False,
            xlim=(0, 20), ylim=(0, 20),
            initial_x=None):
    
    N_enu = len(enu_df)
    
    plt.figure()
   
    # create particles and weights 
    if initial_x is not None:
        particles = create_gaussian_particles(
            mean=initial_x, std=(5, 5, 5), N=N)
    else:
        particles = create_uniform_particles((0,20), (0,20), (0, 6.28), N)
    weights = np.ones(N) / N

    if plot_particles:
        alpha = .20
        if N > 5000:
            alpha *= np.sqrt(5000)/np.sqrt(N)           
        plt.scatter(particles[:, 0], particles[:, 1], 
                    alpha=alpha, color='g')
    
    xs = []
    car_pos = np.array([0., 0.])
    for x in range(iters):
        car_pos += (1, 1)

        # distance from car to each observed point (enu coord) 
        zs = (norm(enu_df - car_pos, axis=1) + 
              (randn(N_enu) * sensor_std_err))

        # move diagonally forward to (x+1, x+1)
        predict(particles, u=u, std=(.2, .05))
        
        # incorporate measurements
        update(particles, weights, z=zs, R=sensor_std_err, 
               enu_df=enu_df)
        
        # resample if too few effective particles
        if neff(weights) < N/2:
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            assert np.allclose(weights, 1/N)
        mu, var = estimate(particles, weights)
        xs.append(mu)

        if plot_particles:
            plt.scatter(particles[:, 0], particles[:, 1], 
                        color='k', marker=',', s=1)
        p1 = plt.scatter(car_pos[0], car_pos[1], marker='+',
                         color='k', s=180, lw=3)
        p2 = plt.scatter(mu[0], mu[1], marker='s', color='r')
    
    xs = np.array(xs)
    #plt.plot(xs[:, 0], xs[:, 1])
    plt.legend([p1, p2], ['Actual', 'PF'], loc=4, numpoints=1)
    plt.xlim(*xlim)
    plt.ylim(*ylim)
    print('final position error, variance:\n\t', mu - np.array([iters, iters]), var)
    plt.show()

from numpy.random import seed
seed(2) 
run_pf1(N=5, plot_particles=False)

1.3887747834072791e-09 3.259683464489998e-09 2.4945478974605282e-09
-1.9690366686868035e-09 -3.1084418537496687e-09 -2.152530781197832e-09


ValueError: operands could not be broadcast together with shapes (1896,3) (2,) 

<Figure size 432x288 with 0 Axes>